In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from unidecode import unidecode

## Step 0 ## 
Preparing the data

In [2]:
myevents = pd.read_csv("events.csv")

In [3]:
mymatches = pd.read_csv("matches.csv")

/tmp/ipykernel_1776/1231547402.py:1: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  mymatches = pd.read_csv("matches.csv")


In [4]:
mymatches

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Dallas,Colorado,6/15/1996,1996,NaN,"9,704",Cotton Bowl,1996 MLS,Regular Season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Colorado,D.C. United,8/29/1996,1996,NaN,"6,368",Mile High Stadium,1996 MLS,Regular Season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,LA Galaxy,New England,8/8/1996,1996,NaN,"10,251",Rose Bowl,1996 MLS,Regular Season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,New England,D.C. United,7/20/1996,1996,NaN,"18,347",Foxboro Stadium,1996 MLS,Regular Season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284,623441.0,Chicago Fire FC,Seattle Sounders FC,"Saturday, July 16",2022,00:00,"13,601","Soldier Field, Chicago",2022 Major League Soccer,Regular Season,...,3.0,6.0,10.0,2.0,4.0,NaN,NaN,NaN,4-2-3-1,4-2-3-1
7285,623442.0,Inter Miami CF,Charlotte FC,"Saturday, July 16",2022,00:00,"12,274","DRV PNK Stadium, Fort Lauderdale, FL",2022 Major League Soccer,Regular Season,...,0.0,7.0,4.0,1.0,3.0,NaN,NaN,NaN,4-2-3-1,4-1-4-1
7286,623443.0,Minnesota United FC,D.C. United,"Saturday, July 16",2022,00:00,"19,718","Allianz Field, Minnesota",2022 Major League Soccer,Regular Season,...,1.0,6.0,4.0,0.0,3.0,NaN,NaN,NaN,4-2-3-1,4-4-2
7287,623445.0,Colorado Rapids,LA Galaxy,"Saturday, July 16",2022,01:00,"17,975","Dick's Sporting Goods Park, Commerce City, CO",2022 Major League Soccer,Regular Season,...,1.0,5.0,6.0,2.0,2.0,NaN,NaN,NaN,4-1-4-1,4-4-2


In [5]:
# Filter matches from 2008 onwards because there are only events recorded from 2008 onwards
mymatches2008 = mymatches[mymatches['year'] >= 2008]

# Select specific columns
selected_columns = mymatches2008.loc[:, 'id':'away_goal_scorers'].columns.tolist() + \
                   mymatches2008.loc[:, 'home_possessionPct':'away_saves'].columns.tolist()

mymatches2008 = mymatches2008[selected_columns]

#mymatches2008.to_csv('mymatches.csv', index=False)
mymatches2008


,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves
2236,237750.0,Columbus Crew SC,Toronto FC,"Saturday, March 29",2008,20:00,"13,843","MAPFRE Stadium, Columbus",2008 USA Major League Soccer,Regular Season 2008,...,1.0,1.0,0.0,0.0,2.0,1.0,3.0,7.0,8.0,5.0
2237,237751.0,Real Salt Lake,Chicago Fire FC,"Saturday, March 29",2008,22:00,"20,272","Rice-Eccles Stadium, Salt Lake City",2008 USA Major League Soccer,Regular Season 2008,...,3.0,1.0,0.0,0.0,1.0,1.0,3.0,3.0,2.0,6.0
2238,237753.0,Sporting Kansas City,DC United,"Saturday, March 29",2008,23:00,"10,385",Community America Ballpark,2008 USA Major League Soccer,Regular Season 2008,...,2.0,2.0,0.0,0.0,1.0,2.0,2.0,6.0,3.0,1.0
2239,238379.0,New England Revolution,Houston Dynamo,"Saturday, March 29",2008,23:30,"11,116",Gillette Stadium,2008 USA Major League Soccer,Regular Season 2008,...,0.0,1.0,0.0,0.0,2.0,1.0,4.0,5.0,5.0,8.0
2240,237754.0,Colorado Rapids,LA Galaxy,"Saturday, March 29",2008,01:30,"18,943","Dick's Sporting Goods Park, Denver",2008 USA Major League Soccer,Regular Season 2008,...,0.0,2.0,1.0,1.0,3.0,4.0,5.0,3.0,9.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284,623441.0,Chicago Fire FC,Seattle Sounders FC,"Saturday, July 16",2022,00:00,"13,601","Soldier Field, Chicago",2022 Major League Soccer,Regular Season,...,0.0,1.0,0.0,0.0,1.0,3.0,6.0,10.0,2.0,4.0
7285,623442.0,Inter Miami CF,Charlotte FC,"Saturday, July 16",2022,00:00,"12,274","DRV PNK Stadium, Fort Lauderdale, FL",2022 Major League Soccer,Regular Season,...,2.0,5.0,0.0,0.0,5.0,0.0,7.0,4.0,1.0,3.0
7286,623443.0,Minnesota United FC,D.C. United,"Saturday, July 16",2022,00:00,"19,718","Allianz Field, Minnesota",2022 Major League Soccer,Regular Season,...,0.0,1.0,0.0,0.0,2.0,1.0,6.0,4.0,0.0,3.0
7287,623445.0,Colorado Rapids,LA Galaxy,"Saturday, July 16",2022,01:00,"17,975","Dick's Sporting Goods Park, Commerce City, CO",2022 Major League Soccer,Regular Season,...,2.0,4.0,0.0,0.0,2.0,1.0,5.0,6.0,2.0,2.0


## Step 1 ##
Finding all events related with penalty calls: 

5202 events of which some entries belong to the same event (refery calling a penalty, the player takes the penalty and the outcome is recorded...) or it might just refer to the penalty spot throughout a play in the game but not referring to an actual penalty.

In [6]:
penalty_events = myevents[myevents['Event'].str.contains('penalty', case=False, na=False)]
penalty_events

,id,Time,Event
1353,237750,44',Chad Marshall commits a foul on Jeff Cunningha...
1354,237750,44',Collin Samuel places the penalty kick with his...
1417,237750,76',Goal!! Alejandro Moreno hits a very good right...
1629,237753,49',That last goal was assisted by Jack Jewsbury (...
1631,237753,49',Goal!! Ivan Trujillo hits a very good right fo...
...,...,...,...
503541,623438,12',Penalty conceded by Julián Araujo (LA Galaxy) ...
503542,623438,13',"Goal! LA Galaxy 0, San Jose Earthquakes 1. Cr..."
503813,623440,78',Penalty Philadelphia Union. Dániel Gazdag draw...
503814,623440,78',Penalty conceded by Henry Kessler (New England...


## Step 2 ##
Find the scored penalties recorded in matches.csv.

Note that if a goal was scored in a penalty, the minute recorded will be written followed by "PEN" such as "58' PEN".

Create a function to extract penalty minutes:

In [7]:
def extract_penalty_minutes(goal_minutes):
    if pd.isna(goal_minutes):
        return []
    penalty_minutes = re.findall(r'(\d+\'(?:\+\d+\')?\sPEN)', goal_minutes)
    penalty_minutes = [minute.replace(" PEN", "") for minute in penalty_minutes]
    return penalty_minutes

In [8]:
mymatches2008['home_penalty_minutes'] = mymatches2008['home_goal_minutes'].apply(extract_penalty_minutes)
mymatches2008['away_penalty_minutes'] = mymatches2008['away_goal_minutes'].apply(extract_penalty_minutes)
mymatches2008

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,home_penalty_minutes,away_penalty_minutes
2236,237750.0,Columbus Crew SC,Toronto FC,"Saturday, March 29",2008,20:00,"13,843","MAPFRE Stadium, Columbus",2008 USA Major League Soccer,Regular Season 2008,...,0.0,0.0,2.0,1.0,3.0,7.0,8.0,5.0,[],[]
2237,237751.0,Real Salt Lake,Chicago Fire FC,"Saturday, March 29",2008,22:00,"20,272","Rice-Eccles Stadium, Salt Lake City",2008 USA Major League Soccer,Regular Season 2008,...,0.0,0.0,1.0,1.0,3.0,3.0,2.0,6.0,[],[]
2238,237753.0,Sporting Kansas City,DC United,"Saturday, March 29",2008,23:00,"10,385",Community America Ballpark,2008 USA Major League Soccer,Regular Season 2008,...,0.0,0.0,1.0,2.0,2.0,6.0,3.0,1.0,[],[]
2239,238379.0,New England Revolution,Houston Dynamo,"Saturday, March 29",2008,23:30,"11,116",Gillette Stadium,2008 USA Major League Soccer,Regular Season 2008,...,0.0,0.0,2.0,1.0,4.0,5.0,5.0,8.0,[],[]
2240,237754.0,Colorado Rapids,LA Galaxy,"Saturday, March 29",2008,01:30,"18,943","Dick's Sporting Goods Park, Denver",2008 USA Major League Soccer,Regular Season 2008,...,1.0,1.0,3.0,4.0,5.0,3.0,9.0,3.0,[58'],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284,623441.0,Chicago Fire FC,Seattle Sounders FC,"Saturday, July 16",2022,00:00,"13,601","Soldier Field, Chicago",2022 Major League Soccer,Regular Season,...,0.0,0.0,1.0,3.0,6.0,10.0,2.0,4.0,[],[]
7285,623442.0,Inter Miami CF,Charlotte FC,"Saturday, July 16",2022,00:00,"12,274","DRV PNK Stadium, Fort Lauderdale, FL",2022 Major League Soccer,Regular Season,...,0.0,0.0,5.0,0.0,7.0,4.0,1.0,3.0,[],[]
7286,623443.0,Minnesota United FC,D.C. United,"Saturday, July 16",2022,00:00,"19,718","Allianz Field, Minnesota",2022 Major League Soccer,Regular Season,...,0.0,0.0,2.0,1.0,6.0,4.0,0.0,3.0,[],[]
7287,623445.0,Colorado Rapids,LA Galaxy,"Saturday, July 16",2022,01:00,"17,975","Dick's Sporting Goods Park, Commerce City, CO",2022 Major League Soccer,Regular Season,...,0.0,0.0,2.0,1.0,5.0,6.0,2.0,2.0,[],[]


In [9]:
home_penalty = mymatches2008.explode('home_penalty_minutes')[['id', 'home_penalty_minutes']].dropna()
home_penalty['Team'] = 'Home'
home_penalty = home_penalty.rename(columns={'home_penalty_minutes': 'Time'})
home_penalty

,id,Time,Team
2240,237754.0,58',Home
2242,237756.0,36',Home
2249,237763.0,28',Home
2251,237765.0,27',Home
2256,237770.0,58',Home
...,...,...,...
7256,623413.0,45'+4',Home
7260,623417.0,10',Home
7260,623417.0,44',Home
7263,623419.0,26',Home


In [10]:
home_penalty_occurrences = mymatches2008['home_penalty_minutes'].apply(lambda x: isinstance(x, list) and len(x) > 0).sum()
home_penalty_occurrences

715

In [11]:
away_penalty = mymatches2008.explode('away_penalty_minutes')[['id', 'away_penalty_minutes']].dropna()
away_penalty['Team'] = 'Away'
away_penalty = away_penalty.rename(columns={'away_penalty_minutes': 'Time'})
away_penalty

,id,Time,Team
2272,237786.0,79',Away
2298,237812.0,54',Away
2328,237840.0,71',Away
2363,251772.0,82',Away
2366,237879.0,72',Away
...,...,...,...
7263,623419.0,67',Away
7265,623421.0,29',Away
7269,623427.0,67',Away
7276,623431.0,26',Away


In [12]:
away_penalty_ocurrences = mymatches2008['away_penalty_minutes'].apply(lambda x: isinstance(x, list) and len(x) > 0).sum()
away_penalty_ocurrences

401

In [13]:
scored_penalty = pd.concat([home_penalty, away_penalty])
scored_penalty

,id,Time,Team
2240,237754.0,58',Home
2242,237756.0,36',Home
2249,237763.0,28',Home
2251,237765.0,27',Home
2256,237770.0,58',Home
...,...,...,...
7263,623419.0,67',Away
7265,623421.0,29',Away
7269,623427.0,67',Away
7276,623431.0,26',Away


In [14]:
scored_penalty['id'].value_counts()

id
359554.0    3
440832.0    3
597916.0    3
289703.0    3
262178.0    3
           ..
440636.0    1
440627.0    1
440626.0    1
440617.0    1
623438.0    1
Name: count, Length: 1039, dtype: int64

When trying to merge the selection of scored penalties with the events corresponding them, due to duplicates in minutes for the same game (id) I encounter some trouble when selecting them back with extra columns. To sort this I remove any duplicate minutes.

In [15]:
scored_penalty_events = pd.merge(scored_penalty, myevents, on=['id', 'Time'], how='inner')

In [16]:
scored_penalty_events

,id,Time,Team,Event
0,237754.0,58',Home,Terry Cooke crosses the ball. Outcome: open play
1,237754.0,58',Home,Abel Xavier commits a foul on Omar Cummings re...
2,237754.0,58',Home,That last goal was assisted by Omar Cummings
3,237754.0,58',Home,Christian Gomez powers the penalty kick with h...
4,237756.0,36',Home,Cuauhtemoc Blanco places the penalty kick with...
...,...,...,...,...
1467,623419.0,67',Away,"Goal! Charlotte FC 2, Nashville SC 1. Hany Mu..."
1468,623421.0,29',Away,"Goal! FC Cincinnati 1, New York Red Bulls 1. ..."
1469,623427.0,67',Away,"Goal! Real Salt Lake 2, Colorado Rapids 1. Di..."
1470,623431.0,26',Away,"Goal! Inter Miami CF 0, Philadelphia Union 1...."


In [17]:
def contains_goal_or_score(event):
    return 'score' in event.lower()

In [18]:
scored_penalty_events['contains_goal_or_score'] = scored_penalty_events['Event'].apply(contains_goal_or_score)
scored_penalty_events = scored_penalty_events.sort_values(by=['id', 'Time', 'contains_goal_or_score'], ascending=[True, True, False])
cleaned_scored_penalty_events = scored_penalty_events.drop_duplicates(subset=['id', 'Time'], keep='first')
cleaned_scored_penalty_events = cleaned_scored_penalty_events.drop(columns=['contains_goal_or_score'])
cleaned_scored_penalty_events

,id,Time,Team,Event
3,237754.0,58',Home,Christian Gomez powers the penalty kick with h...
4,237756.0,36',Home,Cuauhtemoc Blanco places the penalty kick with...
6,237763.0,28',Home,Jack Jewsbury places the penalty kick with his...
7,237765.0,27',Home,Guillermo Barros Schelotto powers the penalty ...
8,237770.0,58',Home,Landon Donovan powers the penalty kick with hi...
...,...,...,...,...
1469,623427.0,67',Away,"Goal! Real Salt Lake 2, Colorado Rapids 1. Di..."
1470,623431.0,26',Away,"Goal! Inter Miami CF 0, Philadelphia Union 1...."
1471,623438.0,13',Away,"Goal! LA Galaxy 0, San Jose Earthquakes 1. Cr..."
949,623440.0,79',Home,"Goal! Philadelphia Union 2, New England Revol..."


We've successfully located 1165 unique penalty events, these events also appear in the matches dataset, ensuring precision in our findings. 

Out of the 5202 entries of penalty events related, we've been able to discard of these 1472 events that can be categorized as scored penalties. Notice that there's more events than actual scored penalties, that's why this procedure is needed.

From the resting 3730 entries, let's try to find the missed penalties:

## Step 3 ##
Finding the rest of penalties events (not scored ones)

From here we can discard the events we found to be scored goals using merge 'left only' for the values that don't appear in our scored penalties event

In [19]:
myevents = pd.read_csv('events.csv')
penalty_events = myevents[myevents['Event'].str.contains('penalty', case=False, na=False)]
penalty_events

,id,Time,Event
1353,237750,44',Chad Marshall commits a foul on Jeff Cunningha...
1354,237750,44',Collin Samuel places the penalty kick with his...
1417,237750,76',Goal!! Alejandro Moreno hits a very good right...
1629,237753,49',That last goal was assisted by Jack Jewsbury (...
1631,237753,49',Goal!! Ivan Trujillo hits a very good right fo...
...,...,...,...
503541,623438,12',Penalty conceded by Julián Araujo (LA Galaxy) ...
503542,623438,13',"Goal! LA Galaxy 0, San Jose Earthquakes 1. Cr..."
503813,623440,78',Penalty Philadelphia Union. Dániel Gazdag draw...
503814,623440,78',Penalty conceded by Henry Kessler (New England...


In [20]:
non_scored_penalties = pd.merge(penalty_events, scored_penalty_events, on=['id', 'Time', 'Event'], how='left', indicator=True)
non_scored_penalties = non_scored_penalties[non_scored_penalties['_merge'] == 'left_only'].drop(columns=['_merge'])
non_scored_penalties = non_scored_penalties.drop(columns=['contains_goal_or_score'])
non_scored_penalties

,id,Time,Event,Team
0,237750,44',Chad Marshall commits a foul on Jeff Cunningha...,NaN
1,237750,44',Collin Samuel places the penalty kick with his...,NaN
2,237750,76',Goal!! Alejandro Moreno hits a very good right...,NaN
3,237753,49',That last goal was assisted by Jack Jewsbury (...,NaN
4,237753,49',Goal!! Ivan Trujillo hits a very good right fo...,NaN
...,...,...,...,...
5196,623431,24',Penalty conceded by Aimé Mabika (Inter Miami C...,NaN
5198,623438,12',Penalty San Jose Earthquakes. Benjamin Kikanov...,NaN
5199,623438,12',Penalty conceded by Julián Araujo (LA Galaxy) ...,NaN
5201,623440,78',Penalty Philadelphia Union. Dániel Gazdag draw...,NaN


In [21]:
missed_penalties = non_scored_penalties[~non_scored_penalties['Event'].str.contains('goal', case=False, na=False)]
missed_penalties

,id,Time,Event,Team
0,237750,44',Chad Marshall commits a foul on Jeff Cunningha...,NaN
15,237756,35',Sainey Nyassi commits a foul on Justin Mapp re...,NaN
32,237759,51',Danny O'Rourke commits a foul on Oscar Echever...,NaN
45,237763,27',Shalrie Joseph commits a foul on Kurt Morsink ...,NaN
51,237765,5',Brad Guzan takes an indirect freekick with his...,NaN
...,...,...,...,...
5196,623431,24',Penalty conceded by Aimé Mabika (Inter Miami C...,NaN
5198,623438,12',Penalty San Jose Earthquakes. Benjamin Kikanov...,NaN
5199,623438,12',Penalty conceded by Julián Araujo (LA Galaxy) ...,NaN
5201,623440,78',Penalty Philadelphia Union. Dániel Gazdag draw...,NaN


In [22]:
def classify_missed_penalty(event):
    event_lower = event.lower()
    if 'save' in event_lower or 'saved' in event_lower:
        return 'Goalkeeper save'
    elif 'missed' in event_lower:
        return 'Missed'
    else:
        return 'NaN'


In [23]:
missed_penalties['Missed_Type'] = missed_penalties['Event'].apply(classify_missed_penalty)
missed_penalties

/tmp/ipykernel_1776/3147478466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missed_penalties['Missed_Type'] = missed_penalties['Event'].apply(classify_missed_penalty)


,id,Time,Event,Team,Missed_Type
0,237750,44',Chad Marshall commits a foul on Jeff Cunningha...,NaN,NaN
15,237756,35',Sainey Nyassi commits a foul on Justin Mapp re...,NaN,NaN
32,237759,51',Danny O'Rourke commits a foul on Oscar Echever...,NaN,NaN
45,237763,27',Shalrie Joseph commits a foul on Kurt Morsink ...,NaN,NaN
51,237765,5',Brad Guzan takes an indirect freekick with his...,NaN,NaN
...,...,...,...,...,...
5196,623431,24',Penalty conceded by Aimé Mabika (Inter Miami C...,NaN,NaN
5198,623438,12',Penalty San Jose Earthquakes. Benjamin Kikanov...,NaN,NaN
5199,623438,12',Penalty conceded by Julián Araujo (LA Galaxy) ...,NaN,NaN
5201,623440,78',Penalty Philadelphia Union. Dániel Gazdag draw...,NaN,NaN


In [24]:
missed_penalties[missed_penalties['Missed_Type'] == 'Goalkeeper save']

,id,Time,Event,Team,Missed_Type
1159,289769,90',Will Hesmer Penalty - Saved,NaN,Goalkeeper save
1160,289769,90',Troy Perkins Penalty - Saved,NaN,Goalkeeper save
1163,289774,59',Pat Noonan Penalty - Saved,NaN,Goalkeeper save
1164,289774,59',Chris Seitz Penalty - Saved,NaN,Goalkeeper save
1181,289856,30',Robbie Findley Penalty - Saved,NaN,Goalkeeper save
...,...,...,...,...,...
5064,623303,35',Penalty saved! Alejandro Pozuelo (Toronto FC) ...,NaN,Goalkeeper save
5117,623356,3',Penalty saved! Chicharito (LA Galaxy) fails to...,NaN,Goalkeeper save
5149,623406,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,NaN,Goalkeeper save
5152,623401,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,NaN,Goalkeeper save


In [25]:
missed_penalties[missed_penalties['Missed_Type'] == 'Goalkeeper save']['id'].value_counts()

id
436948    4
383041    3
314205    3
622709    3
621977    3
         ..
440850    1
440830    1
440827    1
440825    1
623417    1
Name: count, Length: 195, dtype: int64

In [26]:
missed_penalties[missed_penalties['Missed_Type'] == 'Missed']

,id,Time,Event,Team,Missed_Type
365,237898,81',Jaime Moreno places the penalty kick with his ...,NaN,Missed
476,237937,90'+5',Terry Cooke places the penalty kick with his r...,NaN,Missed
586,260862,80',Dwayne De Rosario powers the penalty kick with...,NaN,Missed
765,262182,32',Freddie Ljungberg powers the penalty kick with...,NaN,Missed
835,262211,35',Omar Cummings places the penalty kick with his...,NaN,Missed
...,...,...,...,...,...
4877,598141,40',"Penalty missed! Still San Jose Earthquakes 1,...",NaN,Missed
4905,621977,120',Penalty missed! Bad penalty by Alex Muyl (Nash...,NaN,Missed
4907,621977,120',Penalty missed! Bad penalty by Walker Zimmerma...,NaN,Missed
4968,623207,78',Penalty missed! Bad penalty by Luciano Acosta ...,NaN,Missed


In [27]:
missed_penalties[missed_penalties['Missed_Type'] == 'Missed']['id'].value_counts()

id
383041    4
436948    4
621977    2
528090    2
284964    2
         ..
384638    1
384668    1
384898    1
384704    1
623434    1
Name: count, Length: 106, dtype: int64

In [28]:
all_missed_penalties = missed_penalties[missed_penalties['Missed_Type'] != 'NaN']
all_missed_penalties

,id,Time,Event,Team,Missed_Type
365,237898,81',Jaime Moreno places the penalty kick with his ...,NaN,Missed
476,237937,90'+5',Terry Cooke places the penalty kick with his r...,NaN,Missed
586,260862,80',Dwayne De Rosario powers the penalty kick with...,NaN,Missed
765,262182,32',Freddie Ljungberg powers the penalty kick with...,NaN,Missed
835,262211,35',Omar Cummings places the penalty kick with his...,NaN,Missed
...,...,...,...,...,...
5117,623356,3',Penalty saved! Chicharito (LA Galaxy) fails to...,NaN,Goalkeeper save
5149,623406,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,NaN,Goalkeeper save
5152,623401,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,NaN,Goalkeeper save
5171,623417,34',Penalty saved! Valentín Castellanos (New York ...,NaN,Goalkeeper save


In [29]:
mymatches2008[mymatches2008['id']== 436948]

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,home_penalty_minutes,away_penalty_minutes
4621,436948.0,Portland Timbers,Sporting Kansas City,"Thursday, October 29",2015,02:00,"21,144","Providence Park, Portland",2015 USA Major League Soccer,Knockout - Western Conf,...,0.0,0.0,6.0,4.0,9.0,6.0,1.0,3.0,[],[]


## Step 4 ##
Join scored and non scored penalties together.

Complete our data frame. Fill in the values missing for Team (home/away) and Outcome of the event to complete our dataset:

Add additional columns that would provide relevant information, such as club name, player name, review the stats included in these datasets, and add the ones consider convinient.

In [30]:
merged_df = pd.concat([cleaned_scored_penalty_events, all_missed_penalties])
merged_df

,id,Time,Team,Event,Missed_Type
3,237754.0,58',Home,Christian Gomez powers the penalty kick with h...,NaN
4,237756.0,36',Home,Cuauhtemoc Blanco places the penalty kick with...,NaN
6,237763.0,28',Home,Jack Jewsbury places the penalty kick with his...,NaN
7,237765.0,27',Home,Guillermo Barros Schelotto powers the penalty ...,NaN
8,237770.0,58',Home,Landon Donovan powers the penalty kick with hi...,NaN
...,...,...,...,...,...
5117,623356.0,3',NaN,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save
5149,623406.0,19',NaN,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save
5152,623401.0,45'+3',NaN,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save
5171,623417.0,34',NaN,Penalty saved! Valentín Castellanos (New York ...,Goalkeeper save


In [31]:
merged_df.rename(columns={"Missed_Type": "Outcome"}, inplace=True)
merged_df['Outcome'].fillna('Goal', inplace=True)
merged_df

/tmp/ipykernel_1776/1079964644.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Outcome'].fillna('Goal', inplace=True)


,id,Time,Team,Event,Outcome
3,237754.0,58',Home,Christian Gomez powers the penalty kick with h...,Goal
4,237756.0,36',Home,Cuauhtemoc Blanco places the penalty kick with...,Goal
6,237763.0,28',Home,Jack Jewsbury places the penalty kick with his...,Goal
7,237765.0,27',Home,Guillermo Barros Schelotto powers the penalty ...,Goal
8,237770.0,58',Home,Landon Donovan powers the penalty kick with hi...,Goal
...,...,...,...,...,...
5117,623356.0,3',NaN,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save
5149,623406.0,19',NaN,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save
5152,623401.0,45'+3',NaN,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save
5171,623417.0,34',NaN,Penalty saved! Valentín Castellanos (New York ...,Goalkeeper save


In [32]:
myplayers = pd.read_csv('all_players.csv')

In [33]:
unique_players = myplayers['Player'].unique().tolist()
unique_players = [unidecode(item) for item in unique_players]
unique_players

['Roy Lassiter',
 'Raul Diaz Arce',
 'Eduardo Hurtado',
 'Preki',
 'Brian McBride',
 'Steve Rammel',
 'Vitalis Takawira',
 'Paul Bravo',
 'Jason Kreis',
 'Giovanni Savarese',
 'Pete Marino',
 'Jean Harbor',
 'Joe-Max Moore',
 'Eric Wynalda',
 'Eduardo Espinoza',
 'Mark Chung',
 'Shaun Bartlett',
 'Giuseppe Galderisi',
 'Cobi Jones',
 'Steve Ralston',
 'Marcelo Balboa',
 'Adrian Paz',
 'Alberto Naveda',
 'Harut Karapetyan',
 'Hugo Sanchez',
 'Maurice Johnston',
 'Antony De Avila',
 'Mauricio Cienfuegos',
 'Gerell Elliott',
 'Martin Vasquez',
 'Brian Maisonneuve',
 'John Kerr',
 'A.J. Wood',
 'Steve Trittschuh',
 'Diego Viera',
 'Marco Etcheverry',
 'Carlos Valderrama',
 'Tony Sanneh',
 'Mike Sorber',
 'Darren Sawatzky',
 'Miles Joseph',
 'Brian Haynes',
 'Paul Keegan',
 'Rob Johnson',
 'Marcelo Carrera',
 'Washington Rodriguez',
 'Greg Vanney',
 'Tab Ramos',
 'Roberto Donadoni',
 'John Harkes',
 'Chris Henderson',
 'John Doyle',
 'Ivan McKinley',
 'Welton',
 'Leonel Alvarez',
 'Scott Be

In [34]:
unique_clubs = mymatches2008['home'].unique().tolist()
unique_clubs = [unidecode(item) for item in unique_clubs]
unique_clubs

['Columbus Crew SC',
 'Real Salt Lake',
 'Sporting Kansas City',
 'New England Revolution',
 'Colorado Rapids',
 'FC Dallas',
 'Chicago Fire FC',
 'LA Galaxy',
 'DC United',
 'New York Red Bulls',
 'Chivas USA',
 'Houston Dynamo',
 'San Jose Earthquakes',
 'Toronto FC',
 'Seattle Sounders FC',
 'Philadelphia Union',
 'Vancouver Whitecaps',
 'Portland Timbers',
 'Montreal Impact',
 'Orlando City SC',
 'New York City FC',
 'Atlanta United FC',
 'Minnesota United FC',
 'LAFC',
 'FC Cincinnati',
 'Houston Dynamo FC',
 'Nashville SC',
 'Inter Miami CF',
 'CF Montreal',
 'Columbus Crew',
 'Austin FC',
 'D.C. United',
 'Charlotte FC']

In [35]:
merged_df['Normalized_Event']= merged_df['Event'].apply(unidecode)

In [36]:
merged_df['Club'] = merged_df['Normalized_Event'].apply(lambda x: next((club for club in unique_clubs if club in x), None))
merged_df['Player'] = merged_df['Normalized_Event'].apply(lambda x: next((player for player in unique_players if player in x), None))
merged_df

,id,Time,Team,Event,Outcome,Normalized_Event,Club,Player
3,237754.0,58',Home,Christian Gomez powers the penalty kick with h...,Goal,Christian Gomez powers the penalty kick with h...,None,Christian Gomez
4,237756.0,36',Home,Cuauhtemoc Blanco places the penalty kick with...,Goal,Cuauhtemoc Blanco places the penalty kick with...,None,Cuauhtemoc Blanco
6,237763.0,28',Home,Jack Jewsbury places the penalty kick with his...,Goal,Jack Jewsbury places the penalty kick with his...,None,Jack Jewsbury
7,237765.0,27',Home,Guillermo Barros Schelotto powers the penalty ...,Goal,Guillermo Barros Schelotto powers the penalty ...,None,Guillermo Barros Schelotto
8,237770.0,58',Home,Landon Donovan powers the penalty kick with hi...,Goal,Landon Donovan powers the penalty kick with hi...,None,Landon Donovan
...,...,...,...,...,...,...,...,...
5117,623356.0,3',NaN,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Penalty saved! Chicharito (LA Galaxy) fails to...,LA Galaxy,None
5149,623406.0,19',NaN,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Penalty saved! Pedro Santos (Columbus Crew) fa...,Columbus Crew,Pedro Santos
5152,623401.0,45'+3',NaN,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Minnesota United FC,Emanuel Reynoso
5171,623417.0,34',NaN,Penalty saved! Valentín Castellanos (New York ...,Goalkeeper save,Penalty saved! Valentin Castellanos (New York ...,New York City FC,Valentin Castellanos


In [37]:
num_none_team = merged_df['Team'].isna().sum()
num_none_team

349

In [38]:
num_none_club = merged_df['Club'].isna().sum()
num_none_club

292

In [39]:
num_none_player = merged_df['Player'].isna().sum()
num_none_player

132

In [40]:
def fill_club(row):
    if pd.isna(row['Club']):
        match = mymatches2008[mymatches2008['id'] == row['id']]
        if not match.empty:
            if row['Team'] == 'Home':
                return match['home'].values[0]
            elif row['Team'] == 'Away':
                return match['away'].values[0]
    return row['Club']

In [41]:
merged_df['Club'] = merged_df.apply(fill_club, axis=1)
merged_df

,id,Time,Team,Event,Outcome,Normalized_Event,Club,Player
3,237754.0,58',Home,Christian Gomez powers the penalty kick with h...,Goal,Christian Gomez powers the penalty kick with h...,Colorado Rapids,Christian Gomez
4,237756.0,36',Home,Cuauhtemoc Blanco places the penalty kick with...,Goal,Cuauhtemoc Blanco places the penalty kick with...,Chicago Fire FC,Cuauhtemoc Blanco
6,237763.0,28',Home,Jack Jewsbury places the penalty kick with his...,Goal,Jack Jewsbury places the penalty kick with his...,Sporting Kansas City,Jack Jewsbury
7,237765.0,27',Home,Guillermo Barros Schelotto powers the penalty ...,Goal,Guillermo Barros Schelotto powers the penalty ...,Columbus Crew SC,Guillermo Barros Schelotto
8,237770.0,58',Home,Landon Donovan powers the penalty kick with hi...,Goal,Landon Donovan powers the penalty kick with hi...,LA Galaxy,Landon Donovan
...,...,...,...,...,...,...,...,...
5117,623356.0,3',NaN,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Penalty saved! Chicharito (LA Galaxy) fails to...,LA Galaxy,None
5149,623406.0,19',NaN,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Penalty saved! Pedro Santos (Columbus Crew) fa...,Columbus Crew,Pedro Santos
5152,623401.0,45'+3',NaN,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Minnesota United FC,Emanuel Reynoso
5171,623417.0,34',NaN,Penalty saved! Valentín Castellanos (New York ...,Goalkeeper save,Penalty saved! Valentin Castellanos (New York ...,New York City FC,Valentin Castellanos


In [42]:
num_none_club = merged_df['Club'].isna().sum()
num_none_club

57

In [43]:
def fill_team(row):
    if pd.isna(row['Team']):
        match = mymatches2008[mymatches2008['id'] == row['id']]
        if not match.empty:
            if row['Club'] == match['home'].values[0]:
                return 'Home'
            elif row['Club'] == match['away'].values[0]:
                return 'Away'
    return row['Team']

In [44]:
merged_df['Team'] = merged_df.apply(fill_team, axis=1)
merged_df

,id,Time,Team,Event,Outcome,Normalized_Event,Club,Player
3,237754.0,58',Home,Christian Gomez powers the penalty kick with h...,Goal,Christian Gomez powers the penalty kick with h...,Colorado Rapids,Christian Gomez
4,237756.0,36',Home,Cuauhtemoc Blanco places the penalty kick with...,Goal,Cuauhtemoc Blanco places the penalty kick with...,Chicago Fire FC,Cuauhtemoc Blanco
6,237763.0,28',Home,Jack Jewsbury places the penalty kick with his...,Goal,Jack Jewsbury places the penalty kick with his...,Sporting Kansas City,Jack Jewsbury
7,237765.0,27',Home,Guillermo Barros Schelotto powers the penalty ...,Goal,Guillermo Barros Schelotto powers the penalty ...,Columbus Crew SC,Guillermo Barros Schelotto
8,237770.0,58',Home,Landon Donovan powers the penalty kick with hi...,Goal,Landon Donovan powers the penalty kick with hi...,LA Galaxy,Landon Donovan
...,...,...,...,...,...,...,...,...
5117,623356.0,3',Home,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Penalty saved! Chicharito (LA Galaxy) fails to...,LA Galaxy,None
5149,623406.0,19',Home,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Penalty saved! Pedro Santos (Columbus Crew) fa...,Columbus Crew,Pedro Santos
5152,623401.0,45'+3',Home,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Minnesota United FC,Emanuel Reynoso
5171,623417.0,34',Home,Penalty saved! Valentín Castellanos (New York ...,Goalkeeper save,Penalty saved! Valentin Castellanos (New York ...,New York City FC,Valentin Castellanos


In [45]:
num_none_team = merged_df['Team'].isna().sum()
num_none_team

76

In [46]:
unique_games = merged_df['id'].unique().tolist()
unique_games

[237754.0,
 237756.0,
 237763.0,
 237765.0,
 237770.0,
 237781.0,
 237784.0,
 237786.0,
 237798.0,
 237807.0,
 237811.0,
 237812.0,
 237816.0,
 237840.0,
 237843.0,
 237848.0,
 237864.0,
 237879.0,
 237891.0,
 237919.0,
 237927.0,
 237932.0,
 237938.0,
 237941.0,
 237945.0,
 237951.0,
 237954.0,
 237956.0,
 237957.0,
 237958.0,
 241514.0,
 251772.0,
 258359.0,
 258360.0,
 260859.0,
 260863.0,
 260864.0,
 261938.0,
 261954.0,
 261964.0,
 261967.0,
 261977.0,
 261980.0,
 261981.0,
 261984.0,
 261999.0,
 262002.0,
 262005.0,
 262006.0,
 262016.0,
 262025.0,
 262033.0,
 262038.0,
 262057.0,
 262061.0,
 262062.0,
 262071.0,
 262084.0,
 262152.0,
 262155.0,
 262164.0,
 262178.0,
 262179.0,
 262180.0,
 262186.0,
 262189.0,
 262195.0,
 262202.0,
 262204.0,
 262273.0,
 262274.0,
 267761.0,
 275375.0,
 284748.0,
 284907.0,
 284963.0,
 289680.0,
 289683.0,
 289685.0,
 289691.0,
 289696.0,
 289700.0,
 289703.0,
 289704.0,
 289705.0,
 289706.0,
 289710.0,
 289716.0,
 289734.0,
 289749.0,
 289773.0,

Seems like we are not able to classify all the players because sometimes these players will have nicknames. 

Take for example one of our last entries index=5117, the commentator calls on 'Chicharito' for missing the penalty but this name doesn't exist in our players' list, Chicharito is just a nickname for Javier Hernandez.

In [47]:
'Chicharito' in unique_players

False

In [48]:
'Javier Hernandez' in unique_players

True

In [49]:
unique_players.append('Chicharito')

In [50]:
merged_df['Player'] = merged_df['Normalized_Event'].apply(lambda x: next((player for player in unique_players if player in x), None))

In [51]:
num_none_player = merged_df['Player'].isna().sum()
num_none_player

128

In [52]:
'Valentin Castellanos' in unique_players

True

In [53]:
'Valentín Castellanos' in unique_players

False

Also noticed trouble handling accents in players' names so I standardized both lits of players and clubs, using Unidecode

In [54]:
merged_df['Player'].value_counts()

Player
Chris Wondolowski    30
Landon Donovan       27
Diego Valeri         26
Federico Higuain     25
Josef Martinez       21
                     ..
Benoit Cheyrou        1
Frank Lampard         1
Benjamin Angoua       1
Matteo Mancosu        1
Walker Zimmerman      1
Name: count, Length: 364, dtype: int64

In [55]:
merged_df

,id,Time,Team,Event,Outcome,Normalized_Event,Club,Player
3,237754.0,58',Home,Christian Gomez powers the penalty kick with h...,Goal,Christian Gomez powers the penalty kick with h...,Colorado Rapids,Christian Gomez
4,237756.0,36',Home,Cuauhtemoc Blanco places the penalty kick with...,Goal,Cuauhtemoc Blanco places the penalty kick with...,Chicago Fire FC,Cuauhtemoc Blanco
6,237763.0,28',Home,Jack Jewsbury places the penalty kick with his...,Goal,Jack Jewsbury places the penalty kick with his...,Sporting Kansas City,Jack Jewsbury
7,237765.0,27',Home,Guillermo Barros Schelotto powers the penalty ...,Goal,Guillermo Barros Schelotto powers the penalty ...,Columbus Crew SC,Guillermo Barros Schelotto
8,237770.0,58',Home,Landon Donovan powers the penalty kick with hi...,Goal,Landon Donovan powers the penalty kick with hi...,LA Galaxy,Landon Donovan
...,...,...,...,...,...,...,...,...
5117,623356.0,3',Home,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Penalty saved! Chicharito (LA Galaxy) fails to...,LA Galaxy,Chicharito
5149,623406.0,19',Home,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Penalty saved! Pedro Santos (Columbus Crew) fa...,Columbus Crew,Pedro Santos
5152,623401.0,45'+3',Home,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Minnesota United FC,Emanuel Reynoso
5171,623417.0,34',Home,Penalty saved! Valentín Castellanos (New York ...,Goalkeeper save,Penalty saved! Valentin Castellanos (New York ...,New York City FC,Valentin Castellanos


## Step 5 ## 
Remove penalty shotout events, only in-game penalty events

In [56]:
filtered_matches = mymatches[mymatches['id'].isin(merged_df['id'])]
filtered_matches

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation
2240,237754.0,Colorado Rapids,LA Galaxy,"Saturday, March 29",2008,01:30,"18,943","Dick's Sporting Goods Park, Denver",2008 USA Major League Soccer,Regular Season 2008,...,4.0,5.0,3.0,9.0,3.0,NaN,NaN,NaN,NaN,NaN
2242,237756.0,Chicago Fire FC,New England Revolution,"Thursday, April 3",2008,00:00,"15,553",SeatGeek Stadium,2008 USA Major League Soccer,Regular Season 2008,...,0.0,1.0,10.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN
2249,237763.0,Sporting Kansas City,New England Revolution,"Wednesday, April 9",2008,00:00,"8,096",Community America Ballpark,2008 USA Major League Soccer,Regular Season 2008,...,1.0,10.0,3.0,3.0,8.0,NaN,NaN,NaN,NaN,NaN
2251,237765.0,Columbus Crew SC,Chivas USA,"Saturday, April 12",2008,23:30,"6,733","MAPFRE Stadium, Columbus",2008 USA Major League Soccer,Regular Season 2008,...,1.0,7.0,2.0,2.0,8.0,NaN,NaN,NaN,NaN,NaN
2256,237770.0,LA Galaxy,Toronto FC,"Sunday, April 13",2008,19:00,"20,426","Dignity Health Sports Park, Los Angeles",2008 USA Major League Soccer,Regular Season 2008,...,6.0,3.0,3.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7269,623427.0,Real Salt Lake,Colorado Rapids,"Saturday, July 9",2022,02:00,"20,242","Rio Tinto Stadium, Sandy, UT",2022 Major League Soccer,Regular Season,...,2.0,2.0,6.0,2.0,0.0,NaN,NaN,NaN,4-4-2,4-1-4-1
7274,623434.0,D.C. United,Columbus Crew,"Wednesday, July 13",2022,00:00,"10,128","Audi Field, Washington DC",2022 Major League Soccer,Regular Season,...,4.0,6.0,1.0,1.0,2.0,NaN,NaN,NaN,4-2-3-1,4-2-3-1
7276,623431.0,Inter Miami CF,Philadelphia Union,"Wednesday, July 13",2022,00:00,"10,027","DRV PNK Stadium, Fort Lauderdale, FL",2022 Major League Soccer,Regular Season,...,1.0,4.0,9.0,3.0,2.0,NaN,NaN,NaN,4-2-3-1,4-1-2-1-2
7281,623438.0,LA Galaxy,San Jose Earthquakes,"Wednesday, July 13",2022,02:00,"18,453","Dignity Health Sports Park, Carson",2022 Major League Soccer,Regular Season,...,1.0,11.0,8.0,1.0,6.0,NaN,NaN,NaN,4-4-1-1,4-2-3-1


In [57]:
filtered_matches['shootout'].value_counts()

shootout
False    841
False    400
True      18
True       3
Name: count, dtype: int64

In [58]:
#Normalize values in Regular Season so they don't include a year
filtered_matches['part_of_competition'] = filtered_matches['part_of_competition'].str.replace(r'(\d{4})', '', regex=True).str.strip()
filtered_matches['part_of_competition'].value_counts()

/tmp/ipykernel_1776/3681558901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_matches['part_of_competition'] = filtered_matches['part_of_competition'].str.replace(r'(\d{4})', '', regex=True).str.strip()


part_of_competition
Regular Season                                 1197
Conference Semi-finals                            7
Semi-finals - Eastern Conf                        6
Knockout - Western Conf                           5
MLS is Back - Round of 16                         5
Final                                             4
Knockout - Eastern Conf                           4
Semi-finals - Western Conf                        4
Western Conference Playoffs - First Round         4
Eastern Conference Playoffs - Semifinals          4
MLS is Back - Quarterfinals                       3
Conference Finals                                 3
Finals - Western Conf                             3
Semifinals - Eastern Conf                         3
Eastern Conference Playoffs - First Round         2
Semifinals - Western Conf                         2
MLS is Back - Semifinals                          1
Eastern Conference Playoffs - Play-In Round       1
Eastern Conference Playoffs - Finals        

In [59]:
filtered_matches[filtered_matches['shootout'] == True]['home_goal_minutes']

4621               57':118'
4628             84':90'+1'
4983                    36'
4991                    NaN
5368                    NaN
5776               21':116'
5782    68': 90'+3':97' PEN
6277                    NaN
6280                    79'
6281                    67'
6283                    90'
6521                 5' PEN
6523          4':47':90'+1'
6525                    82'
7000                    NaN
7003                 45'+2'
7004                9':118'
7007                 90'+4'
Name: home_goal_minutes, dtype: object

In [60]:
filtered_matches[filtered_matches['shootout'] == True]['away_goal_minutes']

4621           87':96'
4628               90'
4983               NaN
4991               NaN
5368               NaN
5776          30': 96'
5782           78':93'
6277               NaN
6280               18'
6281           81' PEN
6283               60'
6521                8'
6523    22':34':90'+7'
6525            90'+3'
7000               NaN
7003               38'
7004           3':109'
7007               41'
Name: away_goal_minutes, dtype: object

In [61]:
shootout_ids = filtered_matches[filtered_matches['shootout'] == True]['id'].tolist()
penalty_shootouts = merged_df[merged_df['id'].isin(shootout_ids)]
penalty_shootouts

,id,Time,Team,Event,Outcome,Normalized_Event,Club,Player
699,528152.0,97',Home,"Goal! Seattle Sounders FC 3, Portland Timbers...",Goal,"Goal! Seattle Sounders FC 3, Portland Timbers...",Seattle Sounders FC,Nicolas Lodeiro
1364,571926.0,81',Away,"Goal! Portland Timbers 1, FC Cincinnati 1. Jü...",Goal,"Goal! Portland Timbers 1, FC Cincinnati 1. Ju...",Portland Timbers,Jurgen Locadia
835,588157.0,5',Home,"Goal! Orlando City SC 1, New York City FC 0. ...",Goal,"Goal! Orlando City SC 1, New York City FC 0. ...",Orlando City SC,Nani
2455,436948.0,120',Away,Penalty saved! Dominic Dwyer (Sporting Kansas ...,Goalkeeper save,Penalty saved! Dominic Dwyer (Sporting Kansas ...,Sporting Kansas City,None
2458,436948.0,120',Home,Penalty missed! Bad penalty by Jack Jewsbury (...,Missed,Penalty missed! Bad penalty by Jack Jewsbury (...,Portland Timbers,Jack Jewsbury
2459,436948.0,120',Away,Penalty missed! Bad penalty by Matt Besler (Sp...,Missed,Penalty missed! Bad penalty by Matt Besler (Sp...,Sporting Kansas City,Matt Besler
2462,436948.0,120',Home,Penalty saved! Jorge Flores (Portland Timbers)...,Goalkeeper save,Penalty saved! Jorge Flores (Portland Timbers)...,Portland Timbers,Jorge Flores
2463,436948.0,120',Away,"Penalty missed! Still Portland Timbers 2(3), ...",Missed,"Penalty missed! Still Portland Timbers 2(3), ...",Sporting Kansas City,Kevin Ellis
2468,436948.0,120',Home,Penalty saved! Alvas Powell (Portland Timbers)...,Goalkeeper save,Penalty saved! Alvas Powell (Portland Timbers)...,Portland Timbers,Alvas Powell
2469,436948.0,120',Away,"Penalty missed! Still Portland Timbers 2(5), ...",Missed,"Penalty missed! Still Portland Timbers 2(5), ...",Sporting Kansas City,Saad Abdul-Salaam


In [62]:
#Remove entries that were actual penalties
penalty_shootouts = penalty_shootouts.iloc[3:]
penalty_shootouts = penalty_shootouts[penalty_shootouts["Time"] != "57'"]
penalty_shootouts = penalty_shootouts[penalty_shootouts["Time"] != "79'"]
penalty_shootouts

,id,Time,Team,Event,Outcome,Normalized_Event,Club,Player
2455,436948.0,120',Away,Penalty saved! Dominic Dwyer (Sporting Kansas ...,Goalkeeper save,Penalty saved! Dominic Dwyer (Sporting Kansas ...,Sporting Kansas City,None
2458,436948.0,120',Home,Penalty missed! Bad penalty by Jack Jewsbury (...,Missed,Penalty missed! Bad penalty by Jack Jewsbury (...,Portland Timbers,Jack Jewsbury
2459,436948.0,120',Away,Penalty missed! Bad penalty by Matt Besler (Sp...,Missed,Penalty missed! Bad penalty by Matt Besler (Sp...,Sporting Kansas City,Matt Besler
2462,436948.0,120',Home,Penalty saved! Jorge Flores (Portland Timbers)...,Goalkeeper save,Penalty saved! Jorge Flores (Portland Timbers)...,Portland Timbers,Jorge Flores
2463,436948.0,120',Away,"Penalty missed! Still Portland Timbers 2(3), ...",Missed,"Penalty missed! Still Portland Timbers 2(3), ...",Sporting Kansas City,Kevin Ellis
2468,436948.0,120',Home,Penalty saved! Alvas Powell (Portland Timbers)...,Goalkeeper save,Penalty saved! Alvas Powell (Portland Timbers)...,Portland Timbers,Alvas Powell
2469,436948.0,120',Away,"Penalty missed! Still Portland Timbers 2(5), ...",Missed,"Penalty missed! Still Portland Timbers 2(5), ...",Sporting Kansas City,Saad Abdul-Salaam
2473,436948.0,120',Away,Penalty saved! Jon Kempin (Sporting Kansas Cit...,Goalkeeper save,Penalty saved! Jon Kempin (Sporting Kansas Cit...,Sporting Kansas City,None
2481,437321.0,120',Away,Penalty saved! Andreas Ivanschitz (Seattle Sou...,Goalkeeper save,Penalty saved! Andreas Ivanschitz (Seattle Sou...,Seattle Sounders FC,Andreas Ivanschitz
2483,437321.0,120',Away,Penalty saved! Chad Barrett (Seattle Sounders ...,Goalkeeper save,Penalty saved! Chad Barrett (Seattle Sounders ...,Seattle Sounders FC,Chad Barrett


In [63]:
#Remove shootout penalties from the dataset
regular_merged_df = pd.merge(merged_df, penalty_shootouts, on=['id', 'Time'], how='left', indicator=True)
regular_merged_df = regular_merged_df[regular_merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])
regular_merged_df.rename(columns={'Team_x': 'Team', 'Event_x': 'Event','Outcome_x': 'Outcome', 'Normalized_Event_x': 'Normalized_Event', 'Club_x': 'Club', 'Player_x': 'Player'}, inplace=True)
columns_tokeep = ['id', 'Time', 'Event','Normalized_Event', 'Outcome', 'Team', 'Club', 'Player']
ingame_penalties = regular_merged_df[columns_tokeep]
ingame_penalties

,id,Time,Event,Normalized_Event,Outcome,Team,Club,Player
0,237754.0,58',Christian Gomez powers the penalty kick with h...,Christian Gomez powers the penalty kick with h...,Goal,Home,Colorado Rapids,Christian Gomez
1,237756.0,36',Cuauhtemoc Blanco places the penalty kick with...,Cuauhtemoc Blanco places the penalty kick with...,Goal,Home,Chicago Fire FC,Cuauhtemoc Blanco
2,237763.0,28',Jack Jewsbury places the penalty kick with his...,Jack Jewsbury places the penalty kick with his...,Goal,Home,Sporting Kansas City,Jack Jewsbury
3,237765.0,27',Guillermo Barros Schelotto powers the penalty ...,Guillermo Barros Schelotto powers the penalty ...,Goal,Home,Columbus Crew SC,Guillermo Barros Schelotto
4,237770.0,58',Landon Donovan powers the penalty kick with hi...,Landon Donovan powers the penalty kick with hi...,Goal,Home,LA Galaxy,Landon Donovan
...,...,...,...,...,...,...,...,...
1645,623356.0,3',Penalty saved! Chicharito (LA Galaxy) fails to...,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Home,LA Galaxy,Chicharito
1646,623406.0,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Home,Columbus Crew,Pedro Santos
1647,623401.0,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Home,Minnesota United FC,Emanuel Reynoso
1648,623417.0,34',Penalty saved! Valentín Castellanos (New York ...,Penalty saved! Valentin Castellanos (New York ...,Goalkeeper save,Home,New York City FC,Valentin Castellanos


In [64]:
num_none_player = ingame_penalties['Player'].isna().sum()
num_none_player

125

In [65]:
num_none_player = ingame_penalties['Club'].isna().sum()
num_none_player

57

In [66]:
num_none_player = ingame_penalties['Team'].isna().sum()
num_none_player

73

Need to work on completing the missing entries or remove these rows from dataset

## Add Goalkeepers to our data ##

In [67]:
mygoalkeepers = pd.read_csv('all_goalkeepers.csv')

In [68]:
unique_goalkeepers = mygoalkeepers['Player'].unique().tolist()
unique_goalkeepers

['Mark Dodd',
 'Tony Meola',
 'Mark Dougherty',
 'Jorge Campos',
 'Garth Lagerwey',
 'Chris Woods',
 'Tom Liner',
 'Aidan Heaney',
 'Jeff Causey',
 'David Kramer',
 'Mark Simpson',
 'Jim St. Andre',
 'Dave Salzwedel',
 'Bo Oshoniyi',
 'Pat Harrington',
 'Dusty Hudock',
 'Brad Friedel',
 'David Winner',
 'Mike Ammann',
 'Scott Budnick',
 'Zach Thornton',
 'Jeff Cassar',
 'Mile Miovac',
 'Marcus Hahnemann',
 'Walter Zenga',
 'Scott Garlick',
 'Paul Grafer',
 'Kevin Hartman',
 'Tom Presthus',
 'Chris Snitko',
 'Scott Coufal',
 'Marcus Roy',
 'Ian Feuer',
 'Thomas Ravelli',
 'Juergen Sommer',
 'Doug Petras',
 'Andy Kirk',
 'Matt Reis',
 'Matt Napoleon',
 'Matt Jordan',
 'Bill Andracki',
 'Tim Howard',
 'Randy Dedini',
 'Manuel Carou',
 'Kevin East',
 'Emmett Idzahl',
 'Jeremy Oetman',
 'Reginald Pierre-Jerome',
 'Kevin Rueda',
 'Mike Littman',
 'Mike McGinty',
 'David Allred',
 'Joe Cannon',
 'Greg Sutton',
 'Russell Payne',
 'Cesar Delgado',
 'Kori Hunter',
 'Kurt Kelley',
 'Anthony Latro

In [69]:
ingame_penalties['Goalkeeper'] = ingame_penalties['Normalized_Event'].apply(lambda x: next((goalkeeper for goalkeeper in unique_goalkeepers if goalkeeper in x), None))
ingame_penalties

/tmp/ipykernel_1776/2467776535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingame_penalties['Goalkeeper'] = ingame_penalties['Normalized_Event'].apply(lambda x: next((goalkeeper for goalkeeper in unique_goalkeepers if goalkeeper in x), None))


,id,Time,Event,Normalized_Event,Outcome,Team,Club,Player,Goalkeeper
0,237754.0,58',Christian Gomez powers the penalty kick with h...,Christian Gomez powers the penalty kick with h...,Goal,Home,Colorado Rapids,Christian Gomez,None
1,237756.0,36',Cuauhtemoc Blanco places the penalty kick with...,Cuauhtemoc Blanco places the penalty kick with...,Goal,Home,Chicago Fire FC,Cuauhtemoc Blanco,None
2,237763.0,28',Jack Jewsbury places the penalty kick with his...,Jack Jewsbury places the penalty kick with his...,Goal,Home,Sporting Kansas City,Jack Jewsbury,None
3,237765.0,27',Guillermo Barros Schelotto powers the penalty ...,Guillermo Barros Schelotto powers the penalty ...,Goal,Home,Columbus Crew SC,Guillermo Barros Schelotto,None
4,237770.0,58',Landon Donovan powers the penalty kick with hi...,Landon Donovan powers the penalty kick with hi...,Goal,Home,LA Galaxy,Landon Donovan,None
...,...,...,...,...,...,...,...,...,...
1645,623356.0,3',Penalty saved! Chicharito (LA Galaxy) fails to...,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Home,LA Galaxy,Chicharito,None
1646,623406.0,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Home,Columbus Crew,Pedro Santos,None
1647,623401.0,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Home,Minnesota United FC,Emanuel Reynoso,None
1648,623417.0,34',Penalty saved! Valentín Castellanos (New York ...,Penalty saved! Valentin Castellanos (New York ...,Goalkeeper save,Home,New York City FC,Valentin Castellanos,None


In [70]:
num_none_goalkeeper = ingame_penalties['Goalkeeper'].isna().sum()
num_none_goalkeeper

1436

In [71]:
mymatches_columns = mymatches2008[['id', 'time (utc)', 'attendance', 'part_of_competition', 'year']]
penalty_events_merged_df = ingame_penalties.merge(mymatches_columns, on='id', how='left')
penalty_events_merged_df

,id,Time,Event,Normalized_Event,Outcome,Team,Club,Player,Goalkeeper,time (utc),attendance,part_of_competition,year
0,237754.0,58',Christian Gomez powers the penalty kick with h...,Christian Gomez powers the penalty kick with h...,Goal,Home,Colorado Rapids,Christian Gomez,None,01:30,"18,943",Regular Season 2008,2008.0
1,237756.0,36',Cuauhtemoc Blanco places the penalty kick with...,Cuauhtemoc Blanco places the penalty kick with...,Goal,Home,Chicago Fire FC,Cuauhtemoc Blanco,None,00:00,"15,553",Regular Season 2008,2008.0
2,237763.0,28',Jack Jewsbury places the penalty kick with his...,Jack Jewsbury places the penalty kick with his...,Goal,Home,Sporting Kansas City,Jack Jewsbury,None,00:00,"8,096",Regular Season 2008,2008.0
3,237765.0,27',Guillermo Barros Schelotto powers the penalty ...,Guillermo Barros Schelotto powers the penalty ...,Goal,Home,Columbus Crew SC,Guillermo Barros Schelotto,None,23:30,"6,733",Regular Season 2008,2008.0
4,237770.0,58',Landon Donovan powers the penalty kick with hi...,Landon Donovan powers the penalty kick with hi...,Goal,Home,LA Galaxy,Landon Donovan,None,19:00,"20,426",Regular Season 2008,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,623356.0,3',Penalty saved! Chicharito (LA Galaxy) fails to...,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Home,LA Galaxy,Chicharito,None,22:00,"20,433",Regular Season,2022.0
1460,623406.0,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Home,Columbus Crew,Pedro Santos,None,23:30,"20,469",Regular Season,2022.0
1461,623401.0,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Home,Minnesota United FC,Emanuel Reynoso,None,00:00,NaN,Regular Season,2022.0
1462,623417.0,34',Penalty saved! Valentín Castellanos (New York ...,Penalty saved! Valentin Castellanos (New York ...,Goalkeeper save,Home,New York City FC,Valentin Castellanos,None,17:00,NaN,Regular Season,2022.0


In [72]:
ingame_penalties = penalty_events_merged_df[['id', 'Time', 'Event', 'Normalized_Event', 'Outcome', 'Team', 'Club', 'Player', 'Goalkeeper', 'time (utc)', 'attendance', 'part_of_competition', 'year']]
ingame_penalties

,id,Time,Event,Normalized_Event,Outcome,Team,Club,Player,Goalkeeper,time (utc),attendance,part_of_competition,year
0,237754.0,58',Christian Gomez powers the penalty kick with h...,Christian Gomez powers the penalty kick with h...,Goal,Home,Colorado Rapids,Christian Gomez,None,01:30,"18,943",Regular Season 2008,2008.0
1,237756.0,36',Cuauhtemoc Blanco places the penalty kick with...,Cuauhtemoc Blanco places the penalty kick with...,Goal,Home,Chicago Fire FC,Cuauhtemoc Blanco,None,00:00,"15,553",Regular Season 2008,2008.0
2,237763.0,28',Jack Jewsbury places the penalty kick with his...,Jack Jewsbury places the penalty kick with his...,Goal,Home,Sporting Kansas City,Jack Jewsbury,None,00:00,"8,096",Regular Season 2008,2008.0
3,237765.0,27',Guillermo Barros Schelotto powers the penalty ...,Guillermo Barros Schelotto powers the penalty ...,Goal,Home,Columbus Crew SC,Guillermo Barros Schelotto,None,23:30,"6,733",Regular Season 2008,2008.0
4,237770.0,58',Landon Donovan powers the penalty kick with hi...,Landon Donovan powers the penalty kick with hi...,Goal,Home,LA Galaxy,Landon Donovan,None,19:00,"20,426",Regular Season 2008,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,623356.0,3',Penalty saved! Chicharito (LA Galaxy) fails to...,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Home,LA Galaxy,Chicharito,None,22:00,"20,433",Regular Season,2022.0
1460,623406.0,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Home,Columbus Crew,Pedro Santos,None,23:30,"20,469",Regular Season,2022.0
1461,623401.0,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Home,Minnesota United FC,Emanuel Reynoso,None,00:00,NaN,Regular Season,2022.0
1462,623417.0,34',Penalty saved! Valentín Castellanos (New York ...,Penalty saved! Valentin Castellanos (New York ...,Goalkeeper save,Home,New York City FC,Valentin Castellanos,None,17:00,NaN,Regular Season,2022.0


In [73]:
club_mapping = {
    'TB': 'Tampa Bay Mutiny',
    'DC': 'DC United',
    'LA': 'LA Galaxy',
    'KC': 'Sporting Kansas City',
    'CLB': 'Columbus Crew SC',
    'SJ': 'San Jose Earthquakes',
    'DAL': 'FC Dallas',
    'MET': 'MetroStars',
    'COL': 'Colorado Rapids',
    'NE': 'New England Revolution',
    'MIA': 'Miami Fusion',
    'CHI': 'Chicago Fire FC',
    'RSL': 'Real Salt Lake',
    'TOR': 'Toronto FC',
    'CHV': 'Chivas USA',
    'HOU': 'Houston Dynamo',
    'NY': 'New York Red Bulls',
    'VAN': 'Vancouver Whitecaps',
    'SEA': 'Seattle Sounders FC',
    'PHI': 'Philadelphia Union',
    'MTL': 'Montreal Impact',
    'ORL': 'Orlando City SC',
    'CIN': 'FC Cincinnati',
    'ATL': 'Atlanta United',
    'LAFC': 'LAFC',
    'NSH': 'Nashville SC',
    'MCF': 'Inter Miami CF',
    'NYC': 'New York City FC',
    'MIN': 'Minnesota United FC'
}


In [74]:
myplayers['Full_Club'] = myplayers['Club'].str.strip().map(club_mapping)
myplayers

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season,Full_Club
0,Roy Lassiter,TB,F,30,30,2580,27,4,76,49,...,0.14,20,39,70,2,0,64.47,1996,reg,Tampa Bay Mutiny
1,Raul Diaz Arce,DC,F,28,28,2351,23,2,100,49,...,0.08,32,26,35,6,1,49.00,1996,reg,DC United
2,Eduardo Hurtado,LA,F,26,26,2323,21,7,87,56,...,0.27,48,26,25,5,0,64.37,1996,reg,LA Galaxy
3,Preki,KC,M,32,32,2880,18,13,140,61,...,0.41,26,44,7,3,0,43.57,1996,reg,Sporting Kansas City
4,Brian McBride,CLB,F,28,28,2307,17,3,79,44,...,0.12,21,46,10,0,0,55.70,1996,reg,Columbus Crew SC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15762,Alejandro Pozuelo,NaN,M,1,1,120,0,0,5,0,...,0.00,2,0,0,0,0,0.00,2020,post,NaN
15763,Franco Jara,NaN,F,2,2,210,0,0,5,1,...,0.00,6,2,3,0,0,20.00,2020,post,NaN
15764,Andy Polo,NaN,M,1,1,105,0,0,6,3,...,0.00,0,3,0,0,0,50.00,2020,post,NaN
15765,Cristian Roldan,NaN,M,4,4,360,0,0,6,1,...,0.00,5,11,0,0,0,16.67,2020,post,NaN


In [75]:
num_none_clubs = ingame_penalties['Club'].isna().sum()
num_none_clubs

57

In [76]:
def remove_missing_players(df):
    return df.dropna(subset=['Player'])

In [77]:
ingame_penalties_cleaned = remove_missing_players(ingame_penalties)
ingame_penalties_cleaned

,id,Time,Event,Normalized_Event,Outcome,Team,Club,Player,Goalkeeper,time (utc),attendance,part_of_competition,year
0,237754.0,58',Christian Gomez powers the penalty kick with h...,Christian Gomez powers the penalty kick with h...,Goal,Home,Colorado Rapids,Christian Gomez,None,01:30,"18,943",Regular Season 2008,2008.0
1,237756.0,36',Cuauhtemoc Blanco places the penalty kick with...,Cuauhtemoc Blanco places the penalty kick with...,Goal,Home,Chicago Fire FC,Cuauhtemoc Blanco,None,00:00,"15,553",Regular Season 2008,2008.0
2,237763.0,28',Jack Jewsbury places the penalty kick with his...,Jack Jewsbury places the penalty kick with his...,Goal,Home,Sporting Kansas City,Jack Jewsbury,None,00:00,"8,096",Regular Season 2008,2008.0
3,237765.0,27',Guillermo Barros Schelotto powers the penalty ...,Guillermo Barros Schelotto powers the penalty ...,Goal,Home,Columbus Crew SC,Guillermo Barros Schelotto,None,23:30,"6,733",Regular Season 2008,2008.0
4,237770.0,58',Landon Donovan powers the penalty kick with hi...,Landon Donovan powers the penalty kick with hi...,Goal,Home,LA Galaxy,Landon Donovan,None,19:00,"20,426",Regular Season 2008,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,623303.0,35',Penalty saved! Alejandro Pozuelo (Toronto FC) ...,Penalty saved! Alejandro Pozuelo (Toronto FC) ...,Goalkeeper save,Away,Toronto FC,Alejandro Pozuelo,None,20:00,"17,709",Regular Season,2022.0
1459,623356.0,3',Penalty saved! Chicharito (LA Galaxy) fails to...,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Home,LA Galaxy,Chicharito,None,22:00,"20,433",Regular Season,2022.0
1460,623406.0,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Home,Columbus Crew,Pedro Santos,None,23:30,"20,469",Regular Season,2022.0
1461,623401.0,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Home,Minnesota United FC,Emanuel Reynoso,None,00:00,NaN,Regular Season,2022.0


In [78]:
num_none_clubs = ingame_penalties_cleaned['Club'].isna().sum()
num_none_clubs

42

In [79]:
num_none_teams = ingame_penalties_cleaned['Team'].isna().sum()
num_none_teams

57

In [80]:
player_club_mapping = {
    "Jaime Moreno": "DC United",
    "Terry Cooke": "Colorado Rapids",
    "Dwayne De Rosario": "Houston Dynamo",
    "Fred": "DC United",
    "Omar Cummings": "Colorado Rapids",
    "Conor Casey": "Colorado Rapids",
    "Landon Donovan": "LA Galaxy",
    "Pat Noonan": "Seattle Sounders FC",
    "Brad Davis": "Houston Dynamo",
    "Robbie Findley": "Real Salt Lake",
    "David Ferreira": "FC Dallas",
    "Thierry Henry": "New York Red Bulls",
    "Shalrie Joseph": "New England Revolution",
    "Charlie Davies": "DC United",
    "Mauro Rosales": "Seattle Sounders FC",
    "Chris Wondolowski": "San Jose Earthquakes",
    "Robbie Keane": "LA Galaxy",
    "Sebastian Grazzini": "Chicago Fire FC",
    "Chris Rolfe": "Chicago Fire FC",
    "Mike Magee": "LA Galaxy",
    "Nemanja Nikolic": "Chicago Fire FC",
    "Marco Urena": "LAFC",
    "Bastian Schweinsteiger": "Chicago Fire FC",
    "Bradley Wright-Phillips": "New York Red Bulls",
    "Carlos Vela": "LAFC",
    "Sacha Kljestan": "Chivas USA",
    "Jeff Larentowicz": "Colorado Rapids",
    "Rafael": "New York Red Bulls",
    "Terry Dunfield": "Vancouver Whitecaps",
    "Eric Hassli": "Vancouver Whitecaps",
    "Nicolas Gaitan": "Chicago Fire FC",
    "Marcelino Moreno": "Atlanta United FC",   
}

In [81]:
# Fill in the missing 'Club' values using the mapping
ingame_penalties_cleaned['Club'] = ingame_penalties_cleaned.apply(lambda row: player_club_mapping.get(row['Player'], row['Club']) if pd.isna(row['Club']) else row['Club'], axis=1)


/tmp/ipykernel_1776/1053357383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingame_penalties_cleaned['Club'] = ingame_penalties_cleaned.apply(lambda row: player_club_mapping.get(row['Player'], row['Club']) if pd.isna(row['Club']) else row['Club'], axis=1)


In [82]:
num_none_clubs = ingame_penalties_cleaned['Club'].isna().sum()
num_none_clubs

0

In [83]:
ingame_penalties_cleaned['Team'] = ingame_penalties_cleaned.apply(fill_team, axis=1)

/tmp/ipykernel_1776/3737745220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingame_penalties_cleaned['Team'] = ingame_penalties_cleaned.apply(fill_team, axis=1)


In [84]:
num_none_teams = ingame_penalties_cleaned['Team'].isna().sum()
num_none_teams

25

In [85]:
subset_missing_team = ingame_penalties_cleaned[ingame_penalties_cleaned['Team'].isna()][['id', 'Club', 'Player', 'year']]
subset_missing_team

,id,Club,Player,year
1167,260862.0,Houston Dynamo,Dwayne De Rosario,2009.0
1168,262182.0,DC United,Fred,2009.0
1170,262078.0,DC United,Fred,2009.0
1196,314242.0,DC United,Fred,2011.0
1206,314346.0,Houston Dynamo,Dwayne De Rosario,2011.0
1209,336103.0,Columbus Crew,Milovan Mirosevic,2012.0
1222,357242.0,D.C. United,Chris Pontius,2012.0
1231,359668.0,Columbus Crew,Federico Higuain,2013.0
1235,359576.0,Columbus Crew,Federico Higuain,2013.0
1237,359503.0,D.C. United,Dwayne De Rosario,2013.0


In [86]:
ingame_penalties_cleaned['Club'] = ingame_penalties_cleaned['Club'].replace({
    "D.C. United": "DC United",
    "Columbus Crew": "Columbus Crew SC"
})

/tmp/ipykernel_1776/559958478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingame_penalties_cleaned['Club'] = ingame_penalties_cleaned['Club'].replace({


In [87]:
ids_to_drop = [383041.0, 284964.0, 285079.0, 509724.0]
mypenalties = ingame_penalties_cleaned[~ingame_penalties_cleaned['id'].isin(ids_to_drop)]

In [88]:
num_none_teams = mypenalties['Team'].isna().sum()
num_none_teams

24

In [89]:
subset_missing_team = mypenalties[mypenalties['Team'].isna()][['id', 'Club', 'Player', 'year']]
subset_missing_team

,id,Club,Player,year
1167,260862.0,Houston Dynamo,Dwayne De Rosario,2009.0
1168,262182.0,DC United,Fred,2009.0
1170,262078.0,DC United,Fred,2009.0
1196,314242.0,DC United,Fred,2011.0
1206,314346.0,Houston Dynamo,Dwayne De Rosario,2011.0
1209,336103.0,Columbus Crew SC,Milovan Mirosevic,2012.0
1222,357242.0,DC United,Chris Pontius,2012.0
1231,359668.0,Columbus Crew SC,Federico Higuain,2013.0
1235,359576.0,Columbus Crew SC,Federico Higuain,2013.0
1237,359503.0,DC United,Dwayne De Rosario,2013.0


In [90]:
replacements = {
    ("Dwayne De Rosario", 2009): ("Toronto FC", "Dwayne De Rosario"),
    ("Fred", 2009): ("Seattle Sounders FC", "Freddie Ljungberg"),
    ("Fred", 2011): ("Seattle Sounders FC", "Fredy Montero"),
    ("Dwayne De Rosario", 2011): ("DC United", "Dwayne De Rosario"),
    ("Mike Magee", 2014): ("Chicago Fire FC", "Mike Magee"),
    ("Darwin Quintero", 2020): ("Houston Dynamo FC", "Darwin Quintero"),
    ("Mauro Manotas", 2020): ("Houston Dynamo FC", "Mauro Manotas"),
    ("Jeff Larentowicz", 2014): ("Chicago Fire FC", "Jeff Larentowicz"),
    ("Jeff Larentowicz", 2015): ("Chicago Fire FC", "Jeff Larentowicz")
}


In [91]:
for (player, year), (club, updated_player) in replacements.items():
    mypenalties.loc[(mypenalties["Player"] == player) & (mypenalties["year"] == year), "Club"] = club
    mypenalties.loc[(mypenalties["Player"] == player) & (mypenalties["year"] == year), "Player"] = updated_player

In [92]:
mypenalties['Team'] = mypenalties.apply(fill_team, axis=1)
num_none_teams = mypenalties['Team'].isna().sum()
num_none_teams

/tmp/ipykernel_1776/1782646149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mypenalties['Team'] = mypenalties.apply(fill_team, axis=1)


0

In [93]:
def update_club_name(row):
    match_row = mymatches2008[mymatches2008['id'] == row['id']]
    if row['Team'] == 'Home':
        correct_club = match_row['home'].values[0]
    else:
        correct_club = match_row['away'].values[0]
        
    if row['Club'] != correct_club:
        return correct_club
    return row['Club']

In [94]:
mypenalties['Club'] = mypenalties.apply(update_club_name, axis=1)


/tmp/ipykernel_1776/3664188232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mypenalties['Club'] = mypenalties.apply(update_club_name, axis=1)


In [95]:
num_none_teams = mypenalties['Team'].isna().sum()
num_none_teams

0

In [96]:
matches_relevant = mymatches[['id', 'home', 'away', 'home_starting_1', 'away_starting_1']]


In [97]:
# Merge datasets to fill the 'Goalkeeper' column based on 'Team'
mypenalties = mypenalties.merge(matches_relevant, on='id', how='left')


In [98]:
mypenalties['Goalkeeper'] = mypenalties.apply(
    lambda row: row['away_starting_1'] if row['Team'] == 'Home' else row['home_starting_1'], axis=1
)

In [99]:
mypenalties.drop(columns=['home', 'away', 'home_starting_1', 'away_starting_1'], inplace=True)
mypenalties

,id,Time,Event,Normalized_Event,Outcome,Team,Club,Player,Goalkeeper,time (utc),attendance,part_of_competition,year
0,237754.0,58',Christian Gomez powers the penalty kick with h...,Christian Gomez powers the penalty kick with h...,Goal,Home,Colorado Rapids,Christian Gomez,Steve Cronin,01:30,"18,943",Regular Season 2008,2008.0
1,237756.0,36',Cuauhtemoc Blanco places the penalty kick with...,Cuauhtemoc Blanco places the penalty kick with...,Goal,Home,Chicago Fire FC,Cuauhtemoc Blanco,Matt Reis,00:00,"15,553",Regular Season 2008,2008.0
2,237763.0,28',Jack Jewsbury places the penalty kick with his...,Jack Jewsbury places the penalty kick with his...,Goal,Home,Sporting Kansas City,Jack Jewsbury,Matt Reis,00:00,"8,096",Regular Season 2008,2008.0
3,237765.0,27',Guillermo Barros Schelotto powers the penalty ...,Guillermo Barros Schelotto powers the penalty ...,Goal,Home,Columbus Crew SC,Guillermo Barros Schelotto,Brad Guzan,23:30,"6,733",Regular Season 2008,2008.0
4,237770.0,58',Landon Donovan powers the penalty kick with hi...,Landon Donovan powers the penalty kick with hi...,Goal,Home,LA Galaxy,Landon Donovan,Greg Sutton,19:00,"20,426",Regular Season 2008,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,623303.0,35',Penalty saved! Alejandro Pozuelo (Toronto FC) ...,Penalty saved! Alejandro Pozuelo (Toronto FC) ...,Goalkeeper save,Away,Toronto FC,Alejandro Pozuelo,Thomas Hasal,20:00,"17,709",Regular Season,2022.0
1324,623356.0,3',Penalty saved! Chicharito (LA Galaxy) fails to...,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Home,LA Galaxy,Chicharito,Brad Stuver,22:00,"20,433",Regular Season,2022.0
1325,623406.0,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Home,Columbus Crew,Pedro Santos,Andre Blake,23:30,"20,469",Regular Season,2022.0
1326,623401.0,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Home,Minnesota United FC,Emanuel Reynoso,Zac MacMath,00:00,NaN,Regular Season,2022.0


In [100]:
num_none_goalkeeper = mypenalties['Goalkeeper'].isna().sum()
num_none_goalkeeper

136

In [101]:
subset_missing_goalk = mypenalties[mypenalties['Goalkeeper'].isna()][['id', 'Team', 'Club', 'Goalkeeper', 'year']]
subset_missing_goalk.to_csv("missing goalkeepers.csv", index=True)
subset_missing_goalk

,id,Team,Club,Goalkeeper,year
86,289680.0,Home,Sporting Kansas City,NaN,2010.0
87,289683.0,Home,Houston Dynamo,NaN,2010.0
88,289683.0,Home,Houston Dynamo,NaN,2010.0
89,289685.0,Home,Colorado Rapids,NaN,2010.0
90,289685.0,Away,Chicago Fire FC,NaN,2010.0
...,...,...,...,...,...
1090,314263.0,Home,DC United,NaN,2011.0
1091,314295.0,Home,Vancouver Whitecaps,NaN,2011.0
1092,314328.0,Home,Seattle Sounders FC,NaN,2011.0
1093,314334.0,Home,San Jose Earthquakes,NaN,2011.0


In [102]:
'Greg Sutton' in unique_goalkeepers

True

In [103]:
# Correct the initial entries with the opposite team's goalkeeper
goalkeeper_mapping = {
    289710.0: {'home': 'Jon Busch', 'away': 'Dan Kennedy'},
    289716.0: {'home': 'Stefan Frei', 'away': 'Nick Rimando'},
    289734.0: {'home': 'Dan Kennedy', 'away': 'William Hesmer'},
    289749.0: {'home': 'Dan Kennedy', 'away': 'Troy Perkins'},
    289773.0: {'home': 'Kevin Hartman', 'away': 'Dan Kennedy'},
    289680.0: {'home': 'Troy Perkins', 'away': 'Jimmy Nielsen'},
    289683.0: {'home': 'Nick Rimando', 'away': 'Pat Onstad'},
    289685.0: {'home': 'Andrew Dykstra', 'away': 'Matt Pickens'},
    289691.0: {'home': 'William Hesmer', 'away': 'Kevin Hartman'},
    289696.0: {'home': 'Chris Seitz', 'away': 'Stefan Frei'},
    289700.0: {'home': 'Dario Sala', 'away': 'Bouna Coundoul'},
    289703.0: {'home': 'Stefan Frei', 'away': 'Matt Pickens'},
    289704.0: {'home': 'Kasey Seller', 'away': 'Dario Sala'},
    289705.0: {'home': 'Chris Seitz', 'away': 'Bouna Coundoul'},
    289706.0: {'home': 'Nick Rimando', 'away': 'William Hesmer'},
    289774.0: {'home': 'Kasey Keller', 'away': 'Chris Seitz'},
    289779.0: {'home': 'Eric Kronberg', 'away': 'Kevin Hartman'},
    289784.0: {'home': 'Nick Rimando', 'away': 'Andrew Dykstra'},
    289794.0: {'home': 'Stefan Frei', 'away': 'Chris Seitz'},
    289798.0: {'home': 'Donovan Ricketts', 'away': 'Troy Perkins'},
    289807.0: {'home': 'Kevin Hartman', 'away': 'Matt Pickens'},
    289808.0: {'home': 'Bouna Coundoul', 'away': 'Tally Hall'},
    289813.0: {'home': 'Sean Johnson', 'away': 'Donovan Ricketts'},
    289814.0: {'home': 'Will Hesmer', 'away': 'Chris Seitz'},
    289815.0: {'home': 'Dan Kennedy', 'away': 'Stefan Frei'},
    289819.0: {'home': 'Chris Seitz', 'away': 'Kevin Hartman'},
    289832.0: {'home': 'Bouna Coundoul', 'away': 'Stefan Frei'},
    289834.0: {'home': 'Matt Pickens', 'away': 'William Hesmer'},
    289845.0: {'home': 'Sean Johnson', 'away': 'Kasey Keller'},
    289848.0: {'home': 'Donovan Ricketts', 'away': 'Sean Johnson'},
    289856.0: {'home': 'Nick Rimando', 'away': 'Kasey Keller'},
    289865.0: {'home': 'Sean Johnson', 'away': 'Nick Rimando'},
    289866.0: {'home': 'Kasey Keller', 'away': 'William Hesler'},
    289872.0: {'home': 'Matt Reis', 'away': 'Kevin Hartman'},
    289873.0: {'home': 'Bouna Coundoul', 'away': 'Donovan Ricketts'},
    289875.0: {'home': 'Jon Busch', 'away': 'Stefan Frei'},
    289877.0: {'home': 'William Hesler', 'away': 'Matt Reis'},
    289888.0: {'home': 'Sean Johnson', 'away': 'Kevin Hartman'},
    289895.0: {'home': 'Kasey Keller', 'away': 'Jimmy Nielsen'},
    289906.0: {'home': 'Zach Thornton', 'away': 'Jon Busch'},
    289909.0: {'home': 'Nick Rimando', 'away': 'Matt Pickens'},
    289913.0: {'home': 'Brad Knighton', 'away': 'William Hesler'},
    312220.0: {'home': 'William Hesler', 'away': 'Bill Hamid'},
    312221.0: {'home': 'Kasey Keller', 'away': 'Greg Sutton'},
    312232.0: {'home': 'Bill Hamid', 'away': 'Matt Reis'},
    312233.0: {'home': 'Josh Saunders', 'away': 'Nick Rimando'},
    314106.0: {'home': 'Kevin Hartman', 'away': 'William Hesmer'},
    314113.0: {'home': 'Matt Reis', 'away': 'Jay Nolly'},
    314116.0: {'home': 'Josh Saunders', 'away': 'Bill Hamid'},
    314136.0: {'home': 'Kevin Hartman', 'away': 'Jay Nolly'},
    314137.0: {'home': 'Jimmy Nielsen', 'away': 'Matt Reis'},
    314140.0: {'home': 'Troy Perkins', 'away': 'Josh Saunders'},
    314142.0: {'home': 'Jon Busch', 'away': 'Faryd Mondragon'},
    314143.0: {'home': 'Jay Nolly', 'away': 'William Hesler'},
    314144.0: {'home': 'Jimmy Nielsen', 'away': 'Greg Sutton'},
    314146.0: {'home': 'Stefan Frei', 'away': 'Kasey Keller'},
    314150.0: {'home': 'Kasey Keller', 'away': 'Bill Hamid'},
    314153.0: {'home': 'Tally Hall', 'away': 'Stefan Frei'},
    314154.0: {'home': 'Kasey Keller', 'away': 'William Hesmer'},
    314161.0: {'home': 'Stefan Frei', 'away': 'Kevin Hartman'},
    314164.0: {'home': 'Matt Pickens', 'away': 'Bill Hamid'},
    314165.0: {'home': 'Jay Nolly', 'away': 'Matt Reis'},
    314168.0: {'home': 'Jimmy Nielsen', 'away': 'Josh Saunders'},
    314171.0: {'home': 'Dan Kennedy', 'away': 'Greg Sutton'},
    314173.0: {'home': 'Greg Sutton', 'away': 'Tally Hall'},
    314180.0: {'home': 'Matt Pickens', 'away': 'Bouna Coundoul'},
    314182.0: {'home': 'Tally Hall', 'away': 'Donovan Ricketts'},
    314184.0: {'home': 'Bouna Coundoul', 'away': 'Jay Nolly'},
    314191.0: {'home': 'Bill Hamid', 'away': 'Troy Perkins'},
    314197.0: {'home': 'Faryd Mondragón', 'away': 'Matt Pickens'},
    314202.0: {'home': 'Nick Rimando', 'away': 'William Hesmer'},
    314205.0: {'home': 'Matt Reis', 'away': 'Bouna Coundoul'},
    314207.0: {'home': 'Jay Nolly', 'away': 'Kasey Keller'},
    314219.0: {'home': 'Bill Hamid', 'away': 'Nick Rimando'},
    314221.0: {'home': 'Bouna Coundoul', 'away': 'Troy Perkins'},
    314226.0: {'home': 'Tally Hall', 'away': 'Bill Hamid'},
    314228.0: {'home': 'Joe Cannon', 'away': 'Jimmy Nielsen'},
    314234.0: {'home': 'Joe Cannon', 'away': 'Stefan Frei'},
    314241.0: {'home': 'Matt Reis', 'away': 'Nick Rimando'},
    314242.0: {'home': 'Kasey Keller', 'away': 'Donovan Ricketts'},
    314245.0: {'home': 'William Hesmer', 'away': 'Joe Cannon'},
    314254.0: {'home': 'Kasey Keller', 'away': 'Troy Perkins'},
    314259.0: {'home': 'Troy Perkins', 'away': 'Sean Johnson'},
    314263.0: {'home': 'Matt Reis', 'away': 'Bill Hamid'},
    314271.0: {'home': 'Matt Reis', 'away': 'Matt Pickens'},
    314272.0: {'home': 'Jon Busch', 'away': 'Nick Rimando'},
    314277.0: {'home': 'Donovan Ricketts', 'away': 'Joe Cannon'},
    314278.0: {'home': 'Kasey Keller', 'away': 'Tally Hall'},
    314281.0: {'home': 'Stefan Frei', 'away': 'Troy Perkins'},
    314287.0: {'home': 'Stefan Frei', 'away': 'Bill Hamid'},
    314288.0: {'home': 'Dan Kennedy', 'away': 'Matt Reis'},
    314292.0: {'home': 'Bouna Coundoul', 'away': 'Nick Rimando'},
    314295.0: {'home': 'Sean Johnson', 'away': 'Joe Cannon'},
    314301.0: {'home': 'Kevin Hartman', 'away': 'Faryd Mondragón'},
    314303.0: {'home': 'Matt Pickens', 'away': 'Jon Busch'},
    314308.0: {'home': 'Faryd Mondragón', 'away': 'William Hesmer'},
    314318.0: {'home': 'William Hesmer', 'away': 'Kasey Keller'},
    314302.0: {'home': 'Josh Saunders', 'away': 'Jimmy Nielsen'},
    314326.0: {'home': 'Matt Reis', 'away': 'Faryd Mondragón'},
    314328.0: {'home': 'Nick Rimando', 'away': 'Kasey Keller'},
    314334.0: {'home': 'Sean Johnson', 'away': 'Jon Busch'},
    314336.0: {'home': 'Tally Hall', 'away': 'William Hesmer'},
    314342.0: {'home': 'Jon Busch', 'away': 'Tally Hall'},
    314346.0: {'home': 'Dan Kennedy', 'away': 'Bill Hamid'},
    314352.0: {'home': 'Troy Perkins', 'away': 'Frank Rost'},
    314354.0: {'home': 'Kasey Keller', 'away': 'Joe Cannon'},
    314357.0: {'home': 'Matt Reis', 'away': 'Sean Johnson'},
    314358.0: {'home': 'William Hesmer', 'away': 'Jimmy Nielsen'},
    314256.0: {'home': 'Nick Rimando', 'away': 'Joe Cannon'},
    314375.0: {'home': 'Sean Johnson', 'away': 'Bill Hamid'},
    314388.0: {'home': 'Kevin Hartman', 'away': 'Jon Busch'},
    314392.0: {'home': 'Jimmy Nielsen', 'away': 'Matt Pickens'},
    333527.0: {'home': 'Nick Rimando', 'away': 'Kasey Keller'},
    314393.0: {'home': 'Frank Rost', 'away': 'Josh Saunders'},
    314396.0: {'home': 'Nick Rimando', 'away': 'Josh Saunders'},
}

In [104]:
def update_goalkeeper_corrected(row):
    match_id = row['id']
    team = row['Team']
    if team == 'Home':
        return goalkeeper_mapping.get(match_id, {}).get('home', row['Goalkeeper'])
    elif team == 'Away':
        return goalkeeper_mapping.get(match_id, {}).get('away', row['Goalkeeper'])
    else:
        return row['Goalkeeper']

In [105]:
mypenalties['Goalkeeper'] = mypenalties.apply(update_goalkeeper_corrected, axis=1)
num_none_goalkeeper = mypenalties['Goalkeeper'].isna().sum()
num_none_goalkeeper

0

In [106]:
num_none_attendance = mypenalties['attendance'].isna().sum()
num_none_attendance

247

In [107]:
attendance_map = {
    241514.0: 54045,
    262180.0: 20478,
    262179.0: 10762,
    262204.0: 8580,
    261954.0: 14370,
	262038.0: 12847,
	262084.0: 12563,
    261938.0: 6523,
	284963.0: 25373,
	289680.0: 10385,
	289683.0: 11312,
	289685.0: 12642,
	289691.0: 8145,
	289696.0: 21978,
	289700.0: 18075,
	289703.0: 10945,
    289704.0: 8016,
	289705.0: 13667,
	289706.0: 11298,
    289710.0: 14414,
	289716.0: 14661,
	289734.0: 12669,
	289749.0: 13125,
	289773.0: 13083,
	289774.0: 18755,
	289779.0: 11222,
	289784.0: 13157,
	289794.0: 18493,
	289798.0: 20586,
	289807.0: 11641,
	289808.0: 14183,
	289813.0: 20281,
	289814.0: 17251,
	289815.0: 20060,
	289819.0: 10618,
	289832.0: 21859,
    289834.0: 12152,
	289845.0: 36111,
	289848.0: 20418,
	289856.0: 36078,
	289865.0: 18675,
	289866.0: 14109,
    289872.0: 10617,
	289873.0: 27000,
	289875.0: 20172,
	289877.0: 11589,
	289888.0: 10151,
	289895.0: 10385,
	289906.0: 10351,
	289909.0: 16384,
	289913.0: 14109,
	312220.0: 18132,
	312221.0: 20982,
	312232.0: 12914,
	312233.0: 20507,
	314106.0: 10306,
	314113.0: 20122,
	314116.0: 26622,
	314136.0: 19394,
	314137.0: 11114,
	314140.0: 21376,
	314142.0: 18504,
	314143.0: 12102,
	314144.0: 20982,
	314146.0: 36287,
	314150.0: 12319,
	314153.0: 20086,
	314154.0: 10173,
	314161.0: 8145,
	314164.0: 14238,
	314165.0: 11423,
	314168.0: 24709,
	314171.0: 18029,
	314173.0: 15351,
	314180.0: 15814,
    314182.0: 20276,
	314184.0: 20158,
	314191.0: 18627,
	314197.0: 12411,
	314202.0: 11224,
	314205.0: 20829,
	314207.0: 36502,
	314219.0: 18317,
	314221.0: 18627,
	314226.0: 16064,
	314228.0: 18467,
	314234.0: 19137,
	314241.0: 19324,
	314242.0: 27000,
	314245.0: 20181,
	314254.0: 18627,
	314259.0: 14612,
	314263.0: 13603,
	314271.0: 14233,
	314272.0: 16621,
	314277.0: 27000,
	314278.0: 22039,
	314281.0: 18627,
	314287.0: 14267,
	314288.0: 12247,
	314292.0: 18674,
	314295.0: 19394,
	314301.0: 18544,
	314303.0: 10525,
	314308.0: 12569,
	314318.0: 36364,
	314302.0: 20512,
	314326.0: 20512,
	314328.0: 15074,
	314334.0: 36222,
	314336.0: 9737,
	314342.0: 10362,
	314346.0: 15072,
	314352.0: 12374,
	314354.0: 20636,
	314357.0: 21000,
	314358.0: 14858,
	314256.0: 16063,
	314375.0: 19271,
	314388.0: 17896,
	314392.0: 10525,
	333527.0: 16947,
	314393.0: 36021,
	314396.0: 27000,
	384788.0: 15125,
	384706.0: 20814,
	384871.0: 20814,
	596674.0: 3500,
	596661.0: 7061,
	596660.0: 3112,
	597720.0: 7174,
	597733.0: 1365,
	597740.0: 4875,
	597756.0: 3313,
	597757.0: 7684,
	597762.0: 4935,
	597784.0: 7074,
	597839.0: 3000,
	597854.0: 5000,
	597890.0: 8184,
	597899.0: 1553,
	597933.0: 5000,
	597930.0: 13056,
	597935.0: 7000,
    597967.0: 7000,
	598055.0: 5000,
	597910.0: 19517,
	598108.0: 14309,
	598124.0: 19517,
	598141.0: 15585,
	598144.0: 25117,
	623233.0: 19441,
	623401.0: 19725,
	623417.0: 22502,
	623419.0: 35103
}

In [108]:
def update_attendance(df, attendance_map):
    for entry_id, new_attendance in attendance_map.items():
        df.loc[df["id"] == entry_id, "attendance"] = new_attendance

In [109]:
update_attendance(mypenalties, attendance_map)
num_none_attendance = mypenalties['attendance'].isna().sum()
num_none_attendance

66

In [110]:
mypenalties['attendance'] = mypenalties['attendance'].fillna(0).astype(str).str.replace(',', '').astype(int)

In [111]:
mypenalties_ids = mypenalties['id'].unique().tolist()

In [112]:
dates_ids = mymatches2008[mymatches2008['id'].isin(mypenalties_ids)]
subset = ['id', 'home', 'away', 'date']
dates = dates_ids[subset]
dates

,id,home,away,date
2240,237754.0,Colorado Rapids,LA Galaxy,"Saturday, March 29"
2242,237756.0,Chicago Fire FC,New England Revolution,"Thursday, April 3"
2249,237763.0,Sporting Kansas City,New England Revolution,"Wednesday, April 9"
2251,237765.0,Columbus Crew SC,Chivas USA,"Saturday, April 12"
2256,237770.0,LA Galaxy,Toronto FC,"Sunday, April 13"
...,...,...,...,...
7260,623417.0,New York City FC,New England Revolution,"Saturday, July 9"
7263,623419.0,Charlotte FC,Nashville SC,"Saturday, July 9"
7265,623421.0,FC Cincinnati,New York Red Bulls,"Saturday, July 9"
7269,623427.0,Real Salt Lake,Colorado Rapids,"Saturday, July 9"


In [113]:
mypenalties = pd.merge(mypenalties, dates, on='id')
mypenalties

,id,Time,Event,Normalized_Event,Outcome,Team,Club,Player,Goalkeeper,time (utc),attendance,part_of_competition,year,home,away,date
0,237754.0,58',Christian Gomez powers the penalty kick with h...,Christian Gomez powers the penalty kick with h...,Goal,Home,Colorado Rapids,Christian Gomez,Steve Cronin,01:30,18943,Regular Season 2008,2008.0,Colorado Rapids,LA Galaxy,"Saturday, March 29"
1,237756.0,36',Cuauhtemoc Blanco places the penalty kick with...,Cuauhtemoc Blanco places the penalty kick with...,Goal,Home,Chicago Fire FC,Cuauhtemoc Blanco,Matt Reis,00:00,15553,Regular Season 2008,2008.0,Chicago Fire FC,New England Revolution,"Thursday, April 3"
2,237763.0,28',Jack Jewsbury places the penalty kick with his...,Jack Jewsbury places the penalty kick with his...,Goal,Home,Sporting Kansas City,Jack Jewsbury,Matt Reis,00:00,8096,Regular Season 2008,2008.0,Sporting Kansas City,New England Revolution,"Wednesday, April 9"
3,237765.0,27',Guillermo Barros Schelotto powers the penalty ...,Guillermo Barros Schelotto powers the penalty ...,Goal,Home,Columbus Crew SC,Guillermo Barros Schelotto,Brad Guzan,23:30,6733,Regular Season 2008,2008.0,Columbus Crew SC,Chivas USA,"Saturday, April 12"
4,237770.0,58',Landon Donovan powers the penalty kick with hi...,Landon Donovan powers the penalty kick with hi...,Goal,Home,LA Galaxy,Landon Donovan,Greg Sutton,19:00,20426,Regular Season 2008,2008.0,LA Galaxy,Toronto FC,"Sunday, April 13"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,623303.0,35',Penalty saved! Alejandro Pozuelo (Toronto FC) ...,Penalty saved! Alejandro Pozuelo (Toronto FC) ...,Goalkeeper save,Away,Toronto FC,Alejandro Pozuelo,Thomas Hasal,20:00,17709,Regular Season,2022.0,Vancouver Whitecaps,Toronto FC,"Sunday, May 8"
1324,623356.0,3',Penalty saved! Chicharito (LA Galaxy) fails to...,Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Home,LA Galaxy,Chicharito,Brad Stuver,22:00,20433,Regular Season,2022.0,LA Galaxy,Austin FC,"Sunday, May 29"
1325,623406.0,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Home,Columbus Crew,Pedro Santos,Andre Blake,23:30,20469,Regular Season,2022.0,Columbus Crew,Philadelphia Union,"Sunday, July 3"
1326,623401.0,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Home,Minnesota United FC,Emanuel Reynoso,Zac MacMath,00:00,19725,Regular Season,2022.0,Minnesota United FC,Real Salt Lake,"Sunday, July 3"


In [114]:
custom_order = ['id', 'home', 'away', 'part_of_competition', 'Time', 'Event', 'Outcome', 'Team', 'Club', 'Player', 'Goalkeeper', 'attendance', 'time (utc)', 'date', 'year']
mypenalties = mypenalties[custom_order]
mypenalties

,id,home,away,part_of_competition,Time,Event,Outcome,Team,Club,Player,Goalkeeper,attendance,time (utc),date,year
0,237754.0,Colorado Rapids,LA Galaxy,Regular Season 2008,58',Christian Gomez powers the penalty kick with h...,Goal,Home,Colorado Rapids,Christian Gomez,Steve Cronin,18943,01:30,"Saturday, March 29",2008.0
1,237756.0,Chicago Fire FC,New England Revolution,Regular Season 2008,36',Cuauhtemoc Blanco places the penalty kick with...,Goal,Home,Chicago Fire FC,Cuauhtemoc Blanco,Matt Reis,15553,00:00,"Thursday, April 3",2008.0
2,237763.0,Sporting Kansas City,New England Revolution,Regular Season 2008,28',Jack Jewsbury places the penalty kick with his...,Goal,Home,Sporting Kansas City,Jack Jewsbury,Matt Reis,8096,00:00,"Wednesday, April 9",2008.0
3,237765.0,Columbus Crew SC,Chivas USA,Regular Season 2008,27',Guillermo Barros Schelotto powers the penalty ...,Goal,Home,Columbus Crew SC,Guillermo Barros Schelotto,Brad Guzan,6733,23:30,"Saturday, April 12",2008.0
4,237770.0,LA Galaxy,Toronto FC,Regular Season 2008,58',Landon Donovan powers the penalty kick with hi...,Goal,Home,LA Galaxy,Landon Donovan,Greg Sutton,20426,19:00,"Sunday, April 13",2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,623303.0,Vancouver Whitecaps,Toronto FC,Regular Season,35',Penalty saved! Alejandro Pozuelo (Toronto FC) ...,Goalkeeper save,Away,Toronto FC,Alejandro Pozuelo,Thomas Hasal,17709,20:00,"Sunday, May 8",2022.0
1324,623356.0,LA Galaxy,Austin FC,Regular Season,3',Penalty saved! Chicharito (LA Galaxy) fails to...,Goalkeeper save,Home,LA Galaxy,Chicharito,Brad Stuver,20433,22:00,"Sunday, May 29",2022.0
1325,623406.0,Columbus Crew,Philadelphia Union,Regular Season,19',Penalty saved! Pedro Santos (Columbus Crew) fa...,Goalkeeper save,Home,Columbus Crew,Pedro Santos,Andre Blake,20469,23:30,"Sunday, July 3",2022.0
1326,623401.0,Minnesota United FC,Real Salt Lake,Regular Season,45'+3',Penalty saved! Emanuel Reynoso (Minnesota Unit...,Goalkeeper save,Home,Minnesota United FC,Emanuel Reynoso,Zac MacMath,19725,00:00,"Sunday, July 3",2022.0


In [115]:
mypenalties = mypenalties.sort_values(by='year', ascending=True)

In [116]:
mypenalties.to_csv("penalty_events.csv", index=False)

In [117]:
mygoalk = mygoalkeepers[(mygoalkeepers['Year'] == 2010) |  (mygoalkeepers['Year'] ==2011)]
mygoalk = mygoalk[mygoalk['MINS'] > 0]
mygoalk.to_csv("mygoalkeepers.csv", index=False)

In [154]:
missing_gks_matches = mymatches2008[mymatches2008['id'].isin(missing_goalkeepers_ids)]
subsetcolumns = ['id', 'home', 'away', 'date', 'year']
goalkeepermatches = missing_gks_matches[subsetcolumns]

In [155]:
goalkeepermatches.to_csv('goalkeepermatches.csv', index=False)

In [111]:
missing_attendance_matches = mymatches2008[mymatches2008['id'].isin(missing_attendance_ids)]
subsetcolumns = ['id', 'home', 'away','venue', 'time (utc)', 'date', 'year']
attendancematches = missing_attendance_matches[subsetcolumns]

In [112]:
attendancematches.to_csv('attendancematches.csv', index=False)